In [18]:
    laforge_config = {"root":"../../ubisoft-laforge-animation-dataset/output/BVH/",
            "scale": 0.01,
            "hierachy_remap":[
                ["Hips", ("Hips", None)],
                
                ["LeftUpLeg", ("LeftUpperLeg", "Hips")],
                ["LeftLeg", ("LeftLowerLeg", "LeftUpLeg")],
                ["LeftFoot", ("LeftFoot", "LeftLeg")],
                ["LeftToes", "LeftToes", "LeftFoot"],
                
                ["RightUpLeg",("RightUpperLeg", "Hips")],
                ["RightLeg",("RightLowerLeg", "RightUpLeg")],
                ["RightFoot", ("RightFoot", "RightLeg")],
                ["RightToes", "RightToes", "RightFoot"],

                ["Spine", ("Spine", "Hips")],
                ["Spine1", ("Chest", "Spine")],
                ["Spine2", ("UpperChest", "Spine2")],

                ["LeftShoulder", ("LeftShoulder", "Spine2")],
                ["LeftArm",("LeftUpperArm", "LeftShoulder")],
                ["LeftForeArm", ("LeftLowerArm", "LeftArm")],

                ["Neck", ("Neck", "Spine2")],
                ["Head", ("Head", "Neck")],

                ["RightShoulder", ("RightShoulder", "Spine2")],
                ["RightArm",("RightUpperArm", "RightShoulder")],
                ["RightForeArm", ("RightLowerArm", "RightArm")],           
            ]
}

In [3]:
from bvhtoolbox import Bvh

In [12]:
laforge_fs = 'D:/_GITHUB/ubisoft-laforge-animation-dataset/output/BVH/aiming1_subject1.bvh'
with open(laforge_fs) as f:
    laforge_bvh = Bvh(f.read())

In [49]:
fubot_fs = 'D:/_GITHUB/FuBoT-VR-Dataset/beatsaber/beatsaber_0.bvh'
with open(fubot_fs) as f:
    fubot_bvh = Bvh(f.read())

In [51]:
#collect remapping information
fubot_joint_order = fubot_bvh.get_joints_names()

In [172]:
class bvh_node:
    def __init__(self):
        self.name_laforge = None
        self.name = None
        self.offset = list()
        self.channels = list()
        self.children = list()
        self.parent:bvh_node = None
        self.depth = 0
        self.laforge_indices = list()

        self.endsite_offset = list()
        self.is_endsite = False

    def append_child(self, child_node):
        child_node.parent = self
        self.children.append(child_node)

    def full_name(self):
        if self.parent is None:
            return f'ROOT {self.name}'
        
        return f'JOINT {self.name}'

    def scaled_offset(self, scale):
        return [f'{float(v) * scale:.6f}' for v in self.offset]
        

class bvh_hierarchy:
    def __init__(self, root_node, scale = 0.01):
        self.root:bvh_node = root_node
        self.flat_list = []
        self.flat_list_names = []
        self.__create_flat_list()
        self.scale = scale
        self.spacer = '\t'

    def __create_flat_list(self):
        self.flat_list.clear()
        self.flat_list_names.clear()
        
        self.__flatten_next(self.root)

    def __flatten_next(self, node:bvh_node):
        self.flat_list.append(node)
        self.flat_list_names.append(node.name)

        for child in node.children:
            self.__flatten_next(child)

    def get_node(self, name, laforge_notation = False):
        for n in self.flat_list:
            if (laforge_notation and n.name_laforge == name) or (n.name == name):
                return n

        return None

    def get_hierarchy_string(self):
        s = 'HIERARCHY\n'
        for node in self.flat_list:
            s = self._close_scopes(s, node.depth)
            s += self._get_node_string(node)
        s = self._close_scopes(s)
        return s

    def _get_node_string(self, node:bvh_node):
        if node.is_endsite:
            s = '{0}{1}\n'.format(self.spacer * node.depth, 'End Site')
            s += '{0}{{\n'.format(self.spacer * node.depth)
            s += '{0}{1} {2}\n'.format(self.spacer * (node.depth+1), 'OFFSET', ' '.join(node.scaled_offset(self.scale)))
            s += '{0}}}\n'.format(self.spacer * node.depth)
        else:
            s = '{0}{1}\n'.format(self.spacer * node.depth, node.full_name())
            s += '{0}{{\n'.format(self.spacer * node.depth)
            s += '{0}{1} {2}\n'.format(self.spacer * (node.depth + 1), 'OFFSET', ' '.join(node.scaled_offset(self.scale)))
            s += '{0}{1} {2}\n'.format(self.spacer * (node.depth + 1), 'CHANNELS', ' '.join(node.channels))
                
        return s

    def _close_scopes(self, hierarchy_string, target_depth = 0):
        # Get the last level by counting the spaces to the second to last line break.
        last_depth = hierarchy_string[hierarchy_string[:-1].rfind("\n"):].count(self.spacer)
        diff = last_depth - target_depth
        for depth in range(diff):
            hierarchy_string += '{0}}}\n'.format(self.spacer * (last_depth - depth - 1))
        return hierarchy_string

In [184]:
def remap_node(joint_name, bvh_laforge:Bvh, bvh_fubot:Bvh, lut_name_l2f, parent_node:bvh_node = None, depth=0):
    target_fubot = bvh_fubot.get_joint(joint_name)

    name_laforge = joint_name
    if joint_name in lut_name_l2f:
        name_laforge = lut_name_l2f[joint_name]

    target_laforge = bvh_laforge.get_joint(name_laforge)

    #Create Node
    new_node = bvh_node()
    new_node.name = joint_name
    new_node.name_laforge = name_laforge
    new_node.offset = target_fubot['OFFSET'] #swap skeleton
    new_node.channels = target_laforge['CHANNELS']
    new_node.depth = depth

    if parent_node:
        parent_node.append_child(new_node) #append parent/child

    for end_node in target_fubot.filter('End'):
        new_end = bvh_node()
        new_end.name_laforge = new_end.name = 'End Site'
        new_end.offset = end_node['OFFSET']
        new_end.is_endsite = True
        new_end.depth = depth + 1
        
        new_node.append_child(new_end)

        return new_node

    #Check Children
    for child_node in target_fubot.filter('JOINT'):
        remap_node(child_node.name, bvh_laforge, bvh_fubot, lut_name_l2f, new_node, depth + 1)

    return new_node

def remap_hierarchy(bvh_laforge:Bvh, bvh_fubot:Bvh):
    lut_name_l2f = {
        "Chest":"Spine1",
        "UpperChest":"Spine2",
        "LeftUpperArm":"LeftArm",
        "LeftLowerArm":"LeftForeArm",
        "RightUpperArm":"RightArm",
        "RightLowerArm":"RightForeArm",
        "LeftUpperLeg":"LeftUpLeg",
        "LeftLowerLeg":"LeftLeg",
        "LeftToes":"LeftToe",
        "RightUpperLeg":"RightUpLeg",
        "RightLowerLeg":"RightLeg",
        "RightToes":"RightToe"
    }    


    #Build BVH Hierarchy
    root = remap_node("Hips", bvh_laforge, bvh_fubot, lut_name_l2f, None)
    hierarchy = bvh_hierarchy(root, 1.0)

    return hierarchy

new_hierarchy = remap_hierarchy(laforge_bvh, fubot_bvh)
#print(new_hierarchy.get_hierarchy_string())


In [185]:
def calculate_motion_remap_indices(bvh_laforge:Bvh, bvh_fubot:Bvh, hierarchy:bvh_hierarchy):
    laforge_order = bvh_laforge.get_joints_names()
    lut_motion_indices = list()
    current_idx = 0
    for laf_node in laforge_order:
        n = hierarchy.get_node(laf_node, True)
        n.laforge_indices.clear()

        for i in range(0, int(n.channels[0])):
            n.laforge_indices.append(current_idx)
            current_idx += 1

    fubot_order = bvh_fubot.get_joints_names()
    for fub_node in fubot_order:
        n = hierarchy.get_node(fub_node)
        for i in range(0, len(n.laforge_indices)):
            lut_motion_indices.append(n.laforge_indices[i])

    return lut_motion_indices

def append_remapped_motion(bvh_laforge:Bvh, remap_indices, s):
    s += 'MOTION\n'
    s += f'Frames: {len(bvh_laforge.frames)}\n'
    s += f'Frame Time: {bvh_laforge.frame_time}\n'

    for f in bvh_laforge.frames:
        new_frame = remap_frame(f, remap_indices)
        s += ' '.join(new_frame)
        s += '\n'

    return s

def remap_frame(frame_data, remap_indices, scale = 0.01):
    if len(frame_data) is not len(remap_indices):
        return None

    remapped = [frame_data[index] for index in remap_indices]
    remapped[0] = f'{float(remapped[0])*scale:.6f}' #hardcoded - root translation scale
    remapped[1] = f'{float(remapped[1])*scale:.6f}'
    remapped[2] = f'{float(remapped[2])*scale:.6f}'

    return remapped

lut_indices = calculate_motion_remap_indices(laforge_bvh, fubot_bvh, new_hierarchy)
bvh_out = append_remapped_motion(laforge_bvh, lut_indices, new_hierarchy.get_hierarchy_string())

with open('bvh_out.bvh', 'w') as f:
    f.write(bvh_out)